In [ ]:
import torch
from snn_delays.snn import SNN
from snn_delays.experimental_models.snn_binary import BinarySNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.model_loader import ModelLoader
from snn_delays.utils.train_utils import train, get_device, propagate_batch
from snn_delays.utils.visualization_utils import plot_raster
from snn_delays.utils.test_behavior import tb_save_max_last_acc
import matplotlib.pyplot as plt

device = get_device()

# For reproducibility
torch.manual_seed(10)

ModuleNotFoundError: No module named 'snn_delays'

In [ ]:
### DATASET

# Parameters
dataset = 'stmnist'
total_time = 50
batch_size = 1024

# Resized dataloader
# DATASET
DL = DatasetLoader(dataset=dataset,
                   caching='memory',
                   num_workers=0,
                   batch_size=batch_size,
                   total_time=total_time)

train_loader, test_loader, dataset_dict = DL.get_dataloaders()

[ToFrame(sensor_size=(10, 10, 2), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]


In [ ]:
#snn_full = ModelLoader('full_weights_stmnist50_SNN_l1_1d1.t7_6841726618705036_max_100epoch','exp_snn2chip', batch_size, device, True)
snn_full = ModelLoader('full_weights_(64, 2)stmnist50_l2_1d1.t7_8726618705035972_max_93epoch','exp_snn2chip', batch_size, device, True)
#snn_full = ModelLoader('full_weights_stmnist50_SNN_l1_1d1.t7_874820143884892_max_92epoch','exp_snn2chip', batch_size, device, True)